# EFM Harmonic Density State Validation (N=800)

This notebook performs a high-fidelity simulation (N=800) to validate the density states in the Ehokolo Fluxon Model (EFM), focusing on stability around the hypothesized n'=8 limit.

**Objectives**
- Implement and run the baseline EFM NLKG solver on an 800³ grid using PyTorch on a GPU.
- Simulate field evolution for **specific, individual `n'` values** (e.g., 7.0, 8.0, 9.0, 1.0) to test stability.
- Monitor stability metrics (max amplitude, average density, energy).
- Analyze results to determine stability at high resolution near the proposed limit.
- Ensure reproducibility: document code, parameters, hardware, results saving.

## 1. Setup: Libraries, GPU Check, Drive Mount, Paths

In [1]:
# Clear GPU memory at the start (important for Colab)
import torch
import gc
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

# Install/Import required libraries
# !pip install torch numpy matplotlib tqdm psutil scipy -q
!nvidia-smi

import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import psutil
import time
from datetime import datetime
from google.colab import drive
import os
import gc

# Check GPU and memory
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU VRAM Total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"System RAM Total: {psutil.virtual_memory().total / 1e9:.2f} GB")

# Mount Google Drive for checkpoints and data
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/EFM_Simulations/'
hds_path = os.path.join(base_path, 'HDS_Validation_N800/') # Specific folder for N=800 results
checkpoint_path = os.path.join(hds_path, 'checkpoints/')
data_path = os.path.join(hds_path, 'data/')
plot_path = os.path.join(hds_path, 'plots/')
os.makedirs(checkpoint_path, exist_ok=True)
os.makedirs(data_path, exist_ok=True)
os.makedirs(plot_path, exist_ok=True)
print(f"Paths created/checked:\n Checkpoints: {checkpoint_path}\n Data: {data_path}\n Plots: {plot_path}")

Wed Apr 30 17:09:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P0             64W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## 2. Simulation Parameters (N=800)

In [2]:
# --- Numerical Parameters ---
N = 800  # Grid size (High Resolution)
L = 10.0  # Box size (simulation units)
dx = L / N
dt = 0.00025 # Time step (Smaller dt for N=800 stability, check CFL dx/c)
T_steps = 10000 # Total steps (Adjust as needed, start smaller maybe 5k for tests)
save_interval = 100 # How often to save metrics (more frequent for long runs)
checkpoint_interval = 500 # How often to save full state

# --- Physical Parameters (Baseline NLKG: ∂²ϕ/∂t² - c²∇²ϕ + m²ϕ - gφ³ + ηφ⁵ = 0) ---
c_eff = 1.0    # Effective speed of light (simulation units)
m2 = 1.0     # Mass term squared (m^2)
g = 0.1     # Cubic nonlinearity coefficient (g)
eta = 0.01    # Quintic nonlinearity coefficient (η) - for 3D stability
k_rho = 0.01  # Density coupling constant (rho = k * phi^2)

# --- HDS Parameters ---
rho_ref = 1.5 # Reference density (for n'=1, adjust based on results)
# target_n_primes = [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0] # Full list

# --- Initial Conditions ---
noise_level = 0.01 # Small noise amplitude relative to field amplitude

# --- Precision ---
dtype = torch.float16 # Use float16 for memory

# --- Reporting ---
print("--- Simulation Parameters Set ---")
print(f"Grid Size (N): {N}^3")
print(f"Box Size (L): {L}")
print(f"Spatial Step (dx): {dx}")
print(f"Time Step (dt): {dt}")
print(f"Total Steps (T_steps): {T_steps}")
print(f"Effective c (c_eff): {c_eff}")
print(f"Mass Term (m^2): {m2}")
print(f"Cubic Term (g): {g}")
print(f"Quintic Term (eta): {eta}")
print(f"Density Const (k_rho): {k_rho}")
print(f"Reference Density (rho_ref): {rho_ref}")
print(f"Precision (dtype): {dtype}")
print("-------------------------------")

--- Simulation Parameters Set ---
Grid Size (N): 800^3
Box Size (L): 10.0
Spatial Step (dx): 0.0125
Time Step (dt): 0.00025
Total Steps (T_steps): 10000
Effective c (c_eff): 1.0
Mass Term (m^2): 1.0
Cubic Term (g): 0.1
Quintic Term (eta): 0.01
Density Const (k_rho): 0.01
Reference Density (rho_ref): 1.5
Precision (dtype): torch.float16
-------------------------------


## 3. Helper Functions (Potential, NLKG Derivative, RK4, Metrics)

In [3]:
# Potential function V(phi) = 0.5*m2*phi^2 - 0.25*g*phi^4 + 0.1667*eta*phi^6
# Note: Derivative V'(phi) = m2*phi - g*phi^3 + eta*phi^5 is used in NLKG
def potential(phi, m2_p, g_p, eta_p):
    # Potential calculation might need float32 intermediate for stability/range
    phi_f32 = phi.to(torch.float32) 
    m2_p_f32 = torch.tensor(m2_p, dtype=torch.float32, device=phi.device)
    g_p_f32 = torch.tensor(g_p, dtype=torch.float32, device=phi.device)
    eta_p_f32 = torch.tensor(eta_p, dtype=torch.float32, device=phi.device)
    term2 = 0.5 * m2_p_f32 * phi_f32**2
    term4 = -0.25 * g_p_f32 * phi_f32**4
    term6 = (1.0/6.0) * eta_p_f32 * phi_f32**6
    return (term2 + term4 + term6).to(phi.dtype)

# NLKG derivative calculation with absorbing boundary conditions
def nlkg_derivative(phi, phi_dot, m2_p, g_p, eta_p, c_eff_p, L_p, N_p, dx_p, device_p):
    # Use float32 for intermediate calculations to avoid precision issues
    phi_f32 = phi.to(torch.float32)
    phi_dot_f32 = phi_dot.to(torch.float32)
    dx_p_f32 = torch.tensor(dx_p, dtype=torch.float32, device=device_p)
    
    # Calculate Laplacian using finite differences (2nd order accurate)
    laplacian_f32 = torch.zeros_like(phi_f32)
    for dim in range(3):
        laplacian_f32 += torch.roll(phi_f32, shifts=1, dims=dim)
        laplacian_f32 += torch.roll(phi_f32, shifts=-1, dims=dim)
    laplacian = (laplacian_f32 - 6.0 * phi_f32) / dx_p_f32**2

    # Calculate potential derivative V'(phi) = m2*phi - g*phi^3 + eta*phi^5
    m2_p_f32 = torch.tensor(m2_p, dtype=torch.float32, device=device_p)
    g_p_f32 = torch.tensor(g_p, dtype=torch.float32, device=device_p)
    eta_p_f32 = torch.tensor(eta_p, dtype=torch.float32, device=device_p)
    dV_dphi = m2_p_f32 * phi_f32 - g_p_f32 * phi_f32**3 + eta_p_f32 * phi_f32**5

    # Calculate phi_ddot = c^2 * Laplacian - V'(phi)
    c_eff_p_f32 = torch.tensor(c_eff_p, dtype=torch.float32, device=device_p)
    phi_ddot = c_eff_p_f32**2 * laplacian - dV_dphi

    # Apply absorbing boundary conditions (damping mask to phi_dot)
    boundary_width = int(0.1 * N_p) # 10% boundary width
    damping_factor = 0.05 # Strength of damping (can be tuned)
    mask = torch.ones_like(phi)
    for dim in range(3):
        indices = torch.arange(N_p, device=device_p, dtype=phi.dtype)
        # Create damping profile for one dimension
        damping_profile = torch.ones(N_p, device=device_p, dtype=phi.dtype)
        # Linear ramp down towards boundary
        ramp = torch.linspace(1.0, damping_factor, boundary_width, device=device_p, dtype=phi.dtype)
        damping_profile[:boundary_width] = ramp.flip(dims=[0]) # Flipped for start
        damping_profile[-boundary_width:] = ramp
        
        if dim == 0:
            mask *= damping_profile[:, None, None]
        elif dim == 1:
            mask *= damping_profile[None, :, None]
        else:
            mask *= damping_profile[None, None, :]
            
    # Apply damping only to the velocity term to absorb outgoing waves
    phi_dot_damped = phi_dot_f32 * mask.to(torch.float32)
    
    # Return damped velocity and undamped acceleration, converted back to original dtype
    # Ensure phi_ddot has the same dtype as phi_dot
    return phi_dot_damped.to(phi_dot.dtype), phi_ddot.to(phi_dot.dtype) 

# RK4 integrator
def update_phi_rk4(phi, phi_dot, dt_p, m2_p, g_p, eta_p, c_eff_p, L_p, N_p, dx_p, device_p):
    with torch.no_grad(): # Ensure no gradient tracking during update
        # Store original dtype
        original_dtype = phi.dtype
        # Use float32 for RK4 intermediate steps for better precision
        phi_f32 = phi.to(torch.float32)
        phi_dot_f32 = phi_dot.to(torch.float32)
        dt_p_f32 = torch.tensor(dt_p, dtype=torch.float32, device=device_p)

        # k1
        k1_v, k1_a = nlkg_derivative(phi_f32, phi_dot_f32, m2_p, g_p, eta_p, c_eff_p, L_p, N_p, dx_p, device_p)
        k1_v = k1_v.to(torch.float32); k1_a = k1_a.to(torch.float32)

        # k2
        k2_v, k2_a = nlkg_derivative(phi_f32 + 0.5 * dt_p_f32 * k1_v, phi_dot_f32 + 0.5 * dt_p_f32 * k1_a, m2_p, g_p, eta_p, c_eff_p, L_p, N_p, dx_p, device_p)
        k2_v = k2_v.to(torch.float32); k2_a = k2_a.to(torch.float32)

        # k3
        k3_v, k3_a = nlkg_derivative(phi_f32 + 0.5 * dt_p_f32 * k2_v, phi_dot_f32 + 0.5 * dt_p_f32 * k2_a, m2_p, g_p, eta_p, c_eff_p, L_p, N_p, dx_p, device_p)
        k3_v = k3_v.to(torch.float32); k3_a = k3_a.to(torch.float32)

        # k4
        k4_v, k4_a = nlkg_derivative(phi_f32 + dt_p_f32 * k3_v, phi_dot_f32 + dt_p_f32 * k3_a, m2_p, g_p, eta_p, c_eff_p, L_p, N_p, dx_p, device_p)
        k4_v = k4_v.to(torch.float32); k4_a = k4_a.to(torch.float32)

        # Update
        phi_new_f32 = phi_f32 + (dt_p_f32 / 6.0) * (k1_v + 2.0 * k2_v + 2.0 * k3_v + k4_v)
        phi_dot_new_f32 = phi_dot_f32 + (dt_p_f32 / 6.0) * (k1_a + 2.0 * k2_a + 2.0 * k3_a + k4_a)

        # Clamp to prevent extreme values (adjust if necessary)
        # phi_new_f32 = torch.clamp(phi_new_f32, -100.0, 100.0)
        # phi_dot_new_f32 = torch.clamp(phi_dot_new_f32, -100.0, 100.0)

        # Convert back to original dtype
        phi_new = phi_new_f32.to(original_dtype)
        phi_dot_new = phi_dot_new_f32.to(original_dtype)

        # Memory cleanup for intermediate tensors
        del phi_f32, phi_dot_f32, k1_v, k1_a, k2_v, k2_a, k3_v, k3_a, k4_v, k4_a 
        if torch.cuda.is_available():
             torch.cuda.empty_cache()
        # gc.collect() # Optional: Force GC if memory pressure is extreme

        return phi_new, phi_dot_new

# Function to compute stability metrics
def compute_stability_metrics(phi, phi_dot, k_rho_p, m2_p, g_p, eta_p, c_eff_p, dx_p, device_p):
    with torch.no_grad():
        # Calculate metrics using float32 for stability
        phi_f32 = phi.to(torch.float32)
        phi_dot_f32 = phi_dot.to(torch.float32)
        dx_p_f32 = torch.tensor(dx_p, dtype=torch.float32, device=device_p)
        c_eff_p_f32 = torch.tensor(c_eff_p, dtype=torch.float32, device=device_p)

        # Max Amplitude
        max_amp = torch.max(torch.abs(phi_f32)).item() # .item() extracts scalar, fine on GPU

        # Average Density
        avg_density = k_rho_p * torch.mean(phi_f32**2).item() # torch.mean is fine on GPU

        # Energy components (calculate on GPU)
        kinetic_energy_density = 0.5 * phi_dot_f32**2
        # Calculate gradient using torch.gradient
        grad_phi_tuple = torch.gradient(phi_f32, spacing=dx_p_f32, dim=[0, 1, 2])
        gradient_energy_density = 0.5 * c_eff_p_f32**2 * (grad_phi_tuple[0]**2 + grad_phi_tuple[1]**2 + grad_phi_tuple[2]**2)
        potential_energy_density = potential(phi_f32, m2_p, g_p, eta_p)

        # Total Energy (sum on GPU, multiply by volume element, then get item)
        total_energy_density = kinetic_energy_density + gradient_energy_density + potential_energy_density
        total_energy = torch.sum(total_energy_density).item() * (dx_p_f32.item()**3) # Use .item() for dx

        # Check for NaNs using torch functions before extracting item
        if torch.isnan(phi_f32).any() or torch.isnan(phi_dot_f32).any(): # Check input tensors first
             print("NaN detected in input tensors to metrics computation!")
             return float('nan'), float('nan'), float('nan')

        if not (torch.isfinite(torch.tensor(max_amp)) and torch.isfinite(torch.tensor(avg_density)) and torch.isfinite(torch.tensor(total_energy))):
             print(f"NaN/Inf detected in calculated metrics! MaxAmp: {max_amp}, AvgRho: {avg_density}, TotE: {total_energy}")
             return float('nan'), float('nan'), float('nan')

        # Cleanup intermediate tensors on GPU
        del phi_f32, phi_dot_f32, kinetic_energy_density, grad_phi_tuple, gradient_energy_density, potential_energy_density, total_energy_density
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        return max_amp, avg_density, total_energy

## 4. Simulation Execution (Run Specific n')

Modify the `n_prime_to_run` variable below to the specific n' value you want to simulate in this session. Run this cell to execute the simulation for that single density level.

In [ ]:
# --- SELECT N' VALUE TO RUN --- #
n_prime_to_run = 8.0 # <---- CHANGE THIS VALUE (e.g., 1.0, 7.0, 8.0, 9.0)
# ----------------------------- #

# --- Retrieve Global Parameters ---
# Ensure parameters defined in cell 2 are accessible
N_sim = N 
L_sim = L 
dx_sim = dx 
dt_sim = dt 
T_steps_sim = T_steps
m2_sim = m2 
g_sim = g 
eta_sim = eta 
c_eff_sim = c_eff
k_rho_sim = k_rho 
rho_ref_sim = rho_ref
noise_level_sim = noise_level
global_dtype = dtype # Use the globally set dtype
# -----------------------------------

print(f"\n--- Preparing Simulation for n' = {n_prime_to_run:.2f} --- (N={N_sim})")
sim_start_time = time.time()

# Calculate target density and initial field amplitude
target_rho = rho_ref_sim / n_prime_to_run
initial_amp = np.sqrt(target_rho / k_rho_sim) if k_rho_sim > 0 else 1.0 
print(f" Target Density: {target_rho:.4f}, Initial Amplitude: {initial_amp:.4f}")

# Initialize fields on GPU with target amplitude + noise
print("Initializing fields...")
phi = (torch.ones((N_sim, N_sim, N_sim), device=device, dtype=global_dtype) * initial_amp * 
       (1 + noise_level_sim * torch.randn((N_sim, N_sim, N_sim), device=device, dtype=global_dtype)))
phi_dot = torch.zeros((N_sim, N_sim, N_sim), device=device, dtype=global_dtype)
print(f"Field initialization complete. VRAM used: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

# Track metrics for this run
run_max_amp_history = []
run_avg_rho_history = []
run_energy_history = []
run_stable = True
run_steps_completed = 0

pbar = tqdm(range(T_steps_sim), desc=f"n'={n_prime_to_run:.2f}", leave=True)
for t in pbar:
    # Update fields using RK4
    try:
        phi, phi_dot = update_phi_rk4(phi, phi_dot, dt_sim, m2_sim, g_sim, eta_sim, c_eff_sim, L_sim, N_sim, dx_sim, device)
        run_steps_completed = t + 1
    except RuntimeError as e:
         if "out of memory" in str(e):
             print(f"\n!!! CUDA out of memory at step {t} for n'={n_prime_to_run:.2f}. Stopping run. !!!")
             run_stable = False
             break 
         else:
             print(f"\n!!! Runtime error at step {t} for n'={n_prime_to_run:.2f}: {e} !!!")
             run_stable = False
             break
    except Exception as e: # Catch other potential errors
        print(f"\n!!! Unexpected error at step {t} for n'={n_prime_to_run:.2f}: {e} !!!")
        run_stable = False
        break
        
    # Compute and store metrics periodically
    if t % save_interval == 0 or t == T_steps_sim - 1:
         max_amp, avg_density, total_energy = compute_stability_metrics(
             phi, phi_dot, k_rho_sim, m2_sim, g_sim, eta_sim, c_eff_sim, dx_sim, device
         )

         # Stability check: Check for NaN/Inf or excessively large amplitude
         if not (np.isfinite(max_amp) and np.isfinite(avg_density) and np.isfinite(total_energy)) or max_amp > 50.0: 
             print(f"\n!!! Instability detected at step {t} for n'={n_prime_to_run:.2f} (Max Amp: {max_amp:.2e}, Avg Rho: {avg_density:.4f}, Energy: {total_energy:.2e}) !!!")
             run_stable = False
             # Store last known finite values if possible, otherwise NaN
             run_max_amp_history.append(max_amp if np.isfinite(max_amp) else float('nan'))
             run_avg_rho_history.append(avg_density if np.isfinite(avg_density) else float('nan'))
             run_energy_history.append(total_energy if np.isfinite(total_energy) else float('nan'))
             break # Exit inner loop
         else:
            run_max_amp_history.append(max_amp)
            run_avg_rho_history.append(avg_density)
            run_energy_history.append(total_energy)
            pbar.set_postfix({'Max|φ|': f'{max_amp:.2f}', '<ρ>': f'{avg_density:.4f}', 'E': f'{total_energy:.2e}'})
            
    # Save checkpoint periodically
    if t % checkpoint_interval == 0 and t > 0:
        try:
            chkpt_file = os.path.join(checkpoint_path, f"hds_n{n_prime_to_run:.1f}_step{t}.pt")
            # Ensure tensors are on CPU before saving if issues persist, though torch.save handles GPU tensors
            torch.save({'step': t, 'phi': phi.cpu(), 'phi_dot': phi_dot.cpu()}, chkpt_file)
            phi = phi.to(device) # Move back to GPU if moved
            phi_dot = phi_dot.to(device)
            # print(f"Checkpoint saved: {chkpt_file}") # Optional: uncomment for confirmation
        except Exception as e:
            print(f"Warning: Could not save checkpoint at step {t} for n'={n_prime_to_run:.1f}: {e}")

# --- Post-Simulation --- 
pbar.close() # Close the progress bar

# Store final results for this n'
final_phi_slice_np = None # Initialize
if run_stable and 'phi' in locals() and phi is not None: # Check if phi exists
    try:
        # Move final slice to CPU and convert to NumPy *only when saving*
        final_phi_slice_np = phi[N_sim//2, :, :].cpu().numpy().astype(np.float16) # Convert dtype for saving
    except Exception as e:
        print(f"Error converting final phi slice to numpy for n'={n_prime_to_run}: {e}")
        final_phi_slice_np = None # Set to None if conversion fails

final_results_n_prime = {
    'n_prime': n_prime_to_run,
    'stable': run_stable,
    'steps_completed': run_steps_completed,
    # Convert metric lists to numpy arrays *before* saving
    'max_amp': np.array(run_max_amp_history, dtype=np.float32), # Use float32 for metrics
    'avg_rho': np.array(run_avg_rho_history, dtype=np.float32),
    'energy': np.array(run_energy_history, dtype=np.float32),
    'final_phi_slice': final_phi_slice_np
}

# Save this run's data immediately
try:
    npz_file_path = os.path.join(data_path, f"hds_results_n{n_prime_to_run:.1f}.npz")
    np.savez(npz_file_path, **final_results_n_prime) # Save using **kwargs
    print(f"Saved final results for n' = {n_prime_to_run:.2f} to {npz_file_path}")
except Exception as e:
    print(f"Error saving final results for n' = {n_prime_to_run:.2f}: {e}")

# Clean up GPU memory
if 'phi' in locals(): del phi 
if 'phi_dot' in locals(): del phi_dot 
del run_max_amp_history, run_avg_rho_history, run_energy_history # These are Python lists
if 'final_phi_slice_np' in locals(): del final_phi_slice_np

if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

sim_end_time = time.time()
print(f"--- Simulation for n' = {n_prime_to_run:.2f} finished in {(sim_end_time - sim_start_time) / 60:.2f} minutes ---")

## 5. Analysis and Visualization (Run After Simulations)

Run this section **after** you have completed simulations for the `n'` values you are interested in. It will load the saved `.npz` files and generate the summary plots.

In [ ]:
# --- Analysis & Visualization ---
import glob # To find result files
import matplotlib.pyplot as plt
import numpy as np
import os
from IPython.display import Image, display

# Load all saved results
all_results = {}
# Use parameters defined in the parameter cell
hds_path_an = hds_path # Ensure using the correct path
plot_path_an = plot_path
data_path_an = data_path
rho_ref_an = rho_ref # Use rho_ref defined in params cell
N_an = N # Use N defined in params cell
L_an = L # Use L defined in params cell
save_interval_an = save_interval # Use save_interval

npz_files = glob.glob(os.path.join(data_path_an, "hds_results_n*.npz"))
if not npz_files:
    print("No simulation data files found in:", data_path_an)
else:
    print(f"Found {len(npz_files)} result files. Loading...")
    for f in npz_files:
        try:
            data = np.load(f, allow_pickle=True)
            # Extract n_prime from filename (assuming format hds_results_n<number>.npz)
            n_prime_str = os.path.basename(f).replace("hds_results_n", "").replace(".npz", "")
            n_prime = float(n_prime_str)
            # Load dictionary items correctly using data[key].item() if it's a 0-d array
            all_results[n_prime] = {key: data[key].item() if data[key].ndim == 0 else data[key] for key in data.files} 
            print(f" Loaded data for n'={n_prime:.2f}")
        except Exception as e:
            print(f"Error loading file {f}: {e}")

# --- Plot Stability Metrics vs Time --- 
if all_results:
    plt.figure(figsize=(18, 5))
    n_primes_sorted = sorted(all_results.keys())

    # Max Amplitude
    ax1 = plt.subplot(1, 3, 1)
    for n_prime in n_primes_sorted:
        results = all_results[n_prime]
        steps_axis = np.arange(len(results['max_amp'])) * save_interval_an # Correct x-axis based on save interval
        label_str = f"n'={n_prime:.1f}"
        linestyle = '-' if results['stable'] else ':'
        if not results['stable']:
             label_str += " (Unstable)"
        ax1.plot(steps_axis, results['max_amp'], label=label_str, linestyle=linestyle)
    ax1.set_xlabel(f"Step")
    ax1.set_ylabel("Max |φ|")
    ax1.set_title("Max Amplitude vs Time")
    ax1.set_yscale('log') # Use log scale if amplitudes vary widely
    ax1.grid(True, which="both", ls="--")
    ax1.legend(fontsize='small', ncol=2)

    # Average Density
    ax2 = plt.subplot(1, 3, 2)
    for n_prime in n_primes_sorted:
         results = all_results[n_prime]
         steps_axis = np.arange(len(results['avg_rho'])) * save_interval_an
         target_rho = rho_ref_an / n_prime
         label_str = f"n'={n_prime:.1f}"
         linestyle = '-' if results['stable'] else ':'
         ax2.plot(steps_axis, results['avg_rho'], label=label_str, linestyle=linestyle)
         ax2.axhline(target_rho, linestyle='--', color='gray', alpha=0.5) # Show target density
    ax2.set_xlabel(f"Step")
    ax2.set_ylabel("Average Density <ρ>")
    ax2.set_title("Average Density vs Time")
    ax2.grid(True, which="both", ls="--")
    ax2.legend(fontsize='small', ncol=2)

    # Total Energy
    ax3 = plt.subplot(1, 3, 3)
    for n_prime in n_primes_sorted:
        results = all_results[n_prime]
        steps_axis = np.arange(len(results['energy'])) * save_interval_an
        label_str = f"n'={n_prime:.1f}"
        linestyle = '-' if results['stable'] else ':'
        ax3.plot(steps_axis, results['energy'], label=label_str, linestyle=linestyle)
    ax3.set_xlabel(f"Step")
    ax3.set_ylabel("Total Energy E")
    ax3.set_title("Total Energy vs Time")
    ax3.grid(True, which="both", ls="--")
    ax3.legend(fontsize='small', ncol=2)

    plt.tight_layout()
    plt.savefig(os.path.join(plot_path_an, f"hds_N{N_an}_stability_metrics_vs_time.png"))
    plt.show()
else:
    print("No results loaded to plot metrics vs time.")

# --- Plot Final Stability Indicators vs n' --- 
if all_results:
    final_max_amp = []
    final_avg_rho = []
    is_stable = []
    n_primes_tested = sorted(all_results.keys())

    for n_prime in n_primes_tested:
        results = all_results[n_prime]
        is_stable.append(results['stable'])
        # Use last element if available and stable, otherwise NaN
        final_max_amp.append(results['max_amp'][-1] if results['stable'] and len(results['max_amp']) > 0 else float('nan')) 
        final_avg_rho.append(results['avg_rho'][-1] if results['stable'] and len(results['avg_rho']) > 0 else float('nan'))

    plt.figure(figsize=(12, 5))

    ax1 = plt.subplot(1, 2, 1)
    ax1.plot(n_primes_tested, final_max_amp, 'bo-', label='Final Max |φ|')
    # Mark unstable points
    unstable_x = [n for n, stable in zip(n_primes_tested, is_stable) if not stable]
    # Place marker at a high value if stable max exists, otherwise arbitrary high value
    stable_max_amp = [amp for amp in final_max_amp if np.isfinite(amp)]
    unstable_plot_val = np.nanmax(stable_max_amp)*1.1 if stable_max_amp else 100.0 
    ax1.scatter(unstable_x, [unstable_plot_val]*len(unstable_x), marker='x', color='r', s=100, label='Unstable', zorder=5)
    ax1.set_xlabel("Effective n'")
    ax1.set_ylabel("Final Max |φ|")
    ax1.set_title(f"Final Amplitude vs n' (N={N_an})")
    ax1.set_yscale('log')
    ax1.grid(True, which="both", ls="--")
    ax1.legend()

    ax2 = plt.subplot(1, 2, 2)
    ax2.plot(n_primes_tested, final_avg_rho, 'go-', label='Final <ρ>')
    ax2.plot(n_primes_tested, [rho_ref_an / n_p for n_p in n_primes_tested], 'k--', label='Target ρ_ref/n\'')
    # Mark unstable points
    stable_avg_rho = [rho for rho in final_avg_rho if np.isfinite(rho)]
    unstable_y_rho = [np.nanmax(stable_avg_rho)*1.1 if stable_avg_rho else rho_ref_an*1.1] * len(unstable_x) # Place markers above plot
    ax2.scatter(unstable_x, unstable_y_rho, marker='x', color='r', s=100, label='Unstable', zorder=5)
    ax2.set_xlabel("Effective n'")
    ax2.set_ylabel("Final Average Density <ρ>")
    ax2.set_title(f"Final Average Density vs n' (N={N_an})")
    # ax2.set_yscale('log') # Usually linear scale is better for density comparison
    ax2.grid(True, which="both", ls="--")
    ax2.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(plot_path_an, f"hds_N{N_an}_stability_summary_vs_n_prime.png"))
    plt.show()
else:
     print("No results loaded to plot final stability summary.")

# --- Visualize Final States for Stable Runs --- 
if all_results:
    stable_runs = {n: res for n, res in all_results.items() if res['stable'] and res['final_phi_slice'] is not None and res['final_phi_slice'].size > 0} # Added size check
    num_stable = len(stable_runs)
    if num_stable > 0:
        cols = min(4, num_stable)
        rows = (num_stable + cols - 1) // cols
        plt.figure(figsize=(5*cols, 4*rows))
        stable_count = 0
        for i, n_prime in enumerate(sorted(stable_runs.keys())):
             results = stable_runs[n_prime]
             stable_count += 1
             ax = plt.subplot(rows, cols, stable_count)
             # Ensure data is float32 for imshow if it was saved as float16
             slice_data = results['final_phi_slice'].astype(np.float32) 
             if np.isnan(slice_data).any():
                 print(f"Warning: NaN found in final slice for n'={n_prime:.1f}, skipping visualization.")
                 ax.set_title(f"Final φ (z=0) for n'={n_prime:.1f}\n(Data Contains NaN)")
                 ax.text(0.5, 0.5, 'NaN Data', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
                 continue
             im = ax.imshow(slice_data, extent=[-L_an/2, L_an/2, -L_an/2, L_an/2], cmap='viridis', aspect='auto')
             plt.colorbar(im, ax=ax)
             ax.set_title(f"Final φ (z=0) for n'={n_prime:.1f}")
             ax.set_xlabel("x")
             ax.set_ylabel("y")
        plt.tight_layout()
        plt.savefig(os.path.join(plot_path_an, f"hds_N{N_an}_stable_final_states.png"))
        plt.show()
    else:
        print("No stable final states to visualize.")
else:
    print("No results loaded to visualize final states.")

## 6. Simulation Report (Summary of Findings)

In [ ]:
# --- Final Report Summary ---
print("\n--- Simulation Report (N=800) ---")
print(f"Simulation Timestamp: {datetime.now()}")
# Use parameters loaded/defined in analysis section
print(f"Grid Size (N): {N_an}^3")
print(f"Total Steps per run (intended): {T_steps}") 

if all_results:
    print("Stability Summary:")
    stable_n_primes = []
    unstable_n_primes = []
    for n_prime in sorted(all_results.keys()):
        results = all_results[n_prime]
        status = "Stable" if results['stable'] else "Unstable"
        steps_comp = results.get('steps_completed', T_steps) # Use T_steps if key missing
        if results['stable']:
            stable_n_primes.append(n_prime)
            # Check if metrics arrays are non-empty before accessing last element
            final_amp_str = f"{results['max_amp'][-1]:.2f}" if results['max_amp'].size > 0 else "N/A"
            final_rho_str = f"{results['avg_rho'][-1]:.4f}" if results['avg_rho'].size > 0 else "N/A"
            print(f" n' = {n_prime:.2f}: {status} (Completed: {steps_comp}/{T_steps}, Final Max|φ|={final_amp_str}, Final <ρ>={final_rho_str})")
        else:
            unstable_n_primes.append(n_prime)
            print(f" n' = {n_prime:.2f}: {status} (Completed: {steps_comp}/{T_steps})")

    print(f"\nStable n' values found: {sorted(stable_n_primes)}")
    print(f"Unstable n' values found: {sorted(unstable_n_primes)}")
    print(f"(Stability based on up to {T_steps} steps simulation)")
else:
    print("No simulation results loaded to report.")

print("\nAnalysis plots potentially saved to:", plot_path_an)
print("Data files potentially saved to:", data_path_an)
print("Checkpoints potentially saved to:", checkpoint_path)
print("----------------------------------")